# Custom GLM Model Serving

In [0]:
%pip install polars --quiet
%pip install mlflow --upgrade --pre --quiet
%restart_python

In [0]:
CATALOG = "users"  # TODO
SCHEMA = "your_schema"  # TODO
VOLUME = "custom_glm"  # TODO
EXPERIMENT_NAME = "glm-model-serving"  # TODO

In [0]:
import os
import mlflow
from mlflow.exceptions import RestException

experiment_name = f"{os.getcwd()}/{EXPERIMENT_NAME}"

try:
    mlflow.create_experiment(name=experiment_name)
    print(f"Creating new experiment {experiment_name}")
except RestException as e:
    if "RESOURCE_ALREADY_EXISTS" in str(e):
        mlflow.set_experiment(experiment_name)
        print(f"Experiment {experiment_name} already exists.")

## Train a generic GLM model
Here we train a gerenic GLM model and produce a JSON with the model parameters as output, and then use this as basis for a custom Python model.

In [0]:
import statsmodels.api as sm

# load data
data = sm.datasets.star98.load()
data.exog = sm.add_constant(data.exog, prepend=False)

# train model
glm_binom = sm.GLM(data.endog, data.exog, family=sm.families.Binomial())
res = glm_binom.fit()

# get fitted params
params = {
    "intercept": float(res.params["const"]),
    "coefficients": {
        name: float(val) for name, val in res.params.items() if name != "const"
    },
}
print(params)

## Save model params as JSON

In [0]:
import json

params_json = json.dumps(params)
output_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/glm_params.json"
dbutils.fs.put(output_path, params_json, overwrite=True)

## Get inference dataset

In [0]:
test_df = data.raw_data

## Construct custom model

In [0]:
import json

params_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/glm_params.json"
params_json = dbutils.fs.head(params_path)
model_params = json.loads(params_json)

feature_cols = list(model_params["coefficients"].keys())
print(feature_cols)

In [0]:
import mlflow.pyfunc
import numpy as np
import polars as pl
import pandas as pd


class GLMCustomModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model_params):
        self.model_params = model_params

    def load_context(self, context):
        self.intercept = self.model_params["intercept"]
        self.coefs = self.model_params["coefficients"]
        self.feature_order = list(self.coefs.keys())

    def _score_logit(self, pl_df: pl.DataFrame) -> pl.Series:
        expr = pl.lit(self.intercept)
        for col, coef in self.coefs.items():
            expr = expr + pl.col(col) * coef
        return pl_df.select(expr.alias("logit")).to_series()

    def predict(self, context, model_input: pd.DataFrame) -> pd.DataFrame:
        pl_df = pl.from_pandas(model_input[self.feature_order], rechunk=False)
        logit = self._score_logit(pl_df)
        prob = 1.0 / (1.0 + np.exp(-logit))
        out = prob.to_frame().to_pandas()
        out.columns = ["probability_NABOVE"]
        return out


glm_model = GLMCustomModel(model_params=model_params)
glm_model.load_context(context=None)

preds = glm_model.predict(context=None, model_input=test_df)

display(preds.head())

## Log and register model

In [0]:
with mlflow.start_run(run_name="glm_custom_model") as run:
    mlflow.pyfunc.log_model(
        artifact_path="glm_model",
        artifacts={
            "glm_params": f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/glm_params.json"
        },
        registered_model_name=f"{CATALOG}.{SCHEMA}.custom_glm_model",
        python_model=GLMCustomModel(model_params=model_params),
        pip_requirements=[
            "mlflow",
            "polars==1.30.0",
            "pandas",
            "numpy",
        ],
        input_example=test_df[feature_cols].head(3),
        signature=mlflow.models.infer_signature(
            test_df[feature_cols].head(3),
            pd.DataFrame({"probability_NABOVE": [0.0, 0.0, 0.0]}),
        ),
    )

print("Model logged under run:", run.info.run_id)

## Get GLM params as artifact

In [0]:
import mlflow

logged_models = mlflow.search_logged_models()

filtered_models = logged_models[logged_models["source_run_id"] == run.info.run_id]

logged_model_id = filtered_models["model_id"][0]

# get from model artifact page
artifact_uri = f"dbfs:/databricks/mlflow-tracking/{run.info.experiment_id}/logged_models/{logged_model_id}/artifacts/"
mlflow.artifacts.list_artifacts(artifact_uri)

In [0]:
import json

logged_json_path = mlflow.artifacts.download_artifacts(
    artifact_uri + "/artifacts/glm_params.json"
)
with open(logged_json_path, "r") as f:
    logged_params = json.load(f)

print(logged_params)

In [0]:
import pickle

logged_pickle_path = mlflow.artifacts.download_artifacts(
    artifact_uri + "/python_model.pkl"
)
with open(logged_pickle_path, "rb") as f:
    pickled_model = pickle.load(f)

print(pickled_model.model_params)

## Deploy custom model

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput

w = WorkspaceClient()

ENDPOINT_NAME = "custom-glm-endpoint"  # TODO

served_entity = ServedEntityInput(
    entity_name=f"{CATALOG}.{SCHEMA}.custom_glm_model",
    entity_version=1,
    workload_size="Small",
    scale_to_zero_enabled=True,
    workload_type="CPU",
)

w.serving_endpoints.create_and_wait(
    name=ENDPOINT_NAME,
    config=EndpointCoreConfigInput(served_entities=[served_entity]),
)

## Test model endpoint

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

DATABRICKS_HOST = (
    "https://your-workspace.cloud.databricks.com"  # TODO: Your Databricks Host
)
DATABRICKS_TOKEN = dbutils.secrets.get("your_secret_scope", "your_secret_name")


def create_tf_serving_json(data):
    return {
        "inputs": (
            {name: data[name].tolist() for name in data.keys()}
            if isinstance(data, dict)
            else data.tolist()
        )
    }


def score_model(dataset):
    url = f"{DATABRICKS_HOST}/serving-endpoints/{ENDPOINT_NAME}/invocations"
    headers = {
        "Authorization": f"Bearer {DATABRICKS_TOKEN}",
        "Content-Type": "application/json",
    }
    ds_dict = (
        {"dataframe_split": dataset.to_dict(orient="split")}
        if isinstance(dataset, pd.DataFrame)
        else create_tf_serving_json(dataset)
    )
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method="POST", headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(
            f"Request failed with status {response.status_code}, {response.text}"
        )
    return response.json()


preds = score_model(test_df.head())
preds